# Machine learning model for Forward model probability of trifunctional crosslinking 
## Develop a neural network model that will act as a lookup table and work with IMP
* Generate ground truth dataset to train the NN on 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

* MCMC code to generate ground truth data points

In [ ]:
class generate_synthetic_dataset:
    '''
    1) Create a simulation box
    2) Pick two points randomly with uniform probability
    3) Draw a circle with radius sigma1 and sigma2 around each point
    4) Draw another circle with radius sigma1 + L/2 and sigma2 + L/2 around each point
    5) pick one point within each sphere at random, and compute the distance between the two points, also, 
    compute the volume of the spherical cap of the intersection of the two larger spheres
    6) If the distance between the centers of the spheres is less than sigma1 + sigma2 + L, then 
    7) Store the distance and the volume of the spherical cap intersection as a data point 
    8) Generate the probability and present it to the NN for training   
    '''
    
    def __init__(self, box_size = 400.0, xl_length = 25.0, sigma1 = 3.0, sigma2 = 4.0):
        self.box_size = box_size
        self.xl_length = xl_length
        self.sigma1 = sigma1
        self.sigma2 = sigma2
    
    

        pass